In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from datetime import datetime
from sqlalchemy import create_engine

In [2]:
# read in dow file
file = "dow_copy.csv"

dow_df = pd.read_csv(file)
dow_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,12-Mar-12,"12,959.71","12,920.58","12,976.36","12,919.98",100.00M,0.29%
1,9-Jul-12,"12,736.29","12,772.02","12,772.02","12,686.57",100.15M,-0.28%
2,27-Dec-12,"13,096.31","13,114.97","13,141.74","12,964.08",100.16M,-0.14%
3,21-Jul-15,"17,919.29","18,096.67","18,096.67","17,868.34",100.17M,-1.00%
4,20-Apr-16,"18,096.27","18,059.49","18,167.63","18,031.21",100.21M,0.24%


In [3]:
# convert header UC to LC
dow_df.columns = dow_df.columns.str.lower()
dow_df

,date,price,open,high,low,vol.,change %
0,12-Mar-12,"12,959.71","12,920.58","12,976.36","12,919.98",100.00M,0.29%
1,9-Jul-12,"12,736.29","12,772.02","12,772.02","12,686.57",100.15M,-0.28%
2,27-Dec-12,"13,096.31","13,114.97","13,141.74","12,964.08",100.16M,-0.14%
3,21-Jul-15,"17,919.29","18,096.67","18,096.67","17,868.34",100.17M,-1.00%
4,20-Apr-16,"18,096.27","18,059.49","18,167.63","18,031.21",100.21M,0.24%
...,...,...,...,...,...,...,...
2419,16-Sep-15,"16,739.95","16,599.51","16,755.98","16,593.90",99.62M,0.84%
2420,7-Dec-15,"17,730.51","17,845.49","17,845.49","17,639.25",99.67M,-0.66%
2421,12-Sep-13,"15,300.64","15,327.14","15,345.32","15,283.26",99.76M,-0.17%
2422,8-Feb-13,"13,992.97","13,944.05","14,022.62","13,944.05",99.86M,0.35%


In [4]:
# change header names where needed
dow_df = dow_df.rename(columns={'vol.': 'vol_m', 
                                'change %':'chng',
                                'date': 'dt'})
dow_df.head()

,dt,price,open,high,low,vol_m,chng
0,12-Mar-12,"12,959.71","12,920.58","12,976.36","12,919.98",100.00M,0.29%
1,9-Jul-12,"12,736.29","12,772.02","12,772.02","12,686.57",100.15M,-0.28%
2,27-Dec-12,"13,096.31","13,114.97","13,141.74","12,964.08",100.16M,-0.14%
3,21-Jul-15,"17,919.29","18,096.67","18,096.67","17,868.34",100.17M,-1.00%
4,20-Apr-16,"18,096.27","18,059.49","18,167.63","18,031.21",100.21M,0.24%


In [5]:
# convert dates to pd datetime
dow_df['dt']= pd.to_datetime(dow_df["dt"])
dow_df.head()

,dt,price,open,high,low,vol_m,chng
0,2012-03-12,"12,959.71","12,920.58","12,976.36","12,919.98",100.00M,0.29%
1,2012-07-09,"12,736.29","12,772.02","12,772.02","12,686.57",100.15M,-0.28%
2,2012-12-27,"13,096.31","13,114.97","13,141.74","12,964.08",100.16M,-0.14%
3,2015-07-21,"17,919.29","18,096.67","18,096.67","17,868.34",100.17M,-1.00%
4,2016-04-20,"18,096.27","18,059.49","18,167.63","18,031.21",100.21M,0.24%


In [6]:
dow_df.replace(',','', regex=True, inplace=True)
dow_df['vol_m'].replace('M','', regex=True, inplace=True)
dow_df.replace('%','', regex=True, inplace=True)

In [9]:
dow_df['price'] = dow_df['price'].astype(float)
dow_df['open'] = dow_df['open'].astype(float)
dow_df['high'] = dow_df['high'].astype(float)
dow_df['low'] = dow_df['low'].astype(float)
dow_df['vol_m'] = dow_df['vol_m'].astype(float)
dow_df['chng'] = dow_df['chng'].astype(float)
dow_df.head()

,dt,price,open,high,low,vol_m,chng
0,2012-03-12,12959.71,12920.58,12976.36,12919.98,100.00,0.29
1,2012-07-09,12736.29,12772.02,12772.02,12686.57,100.15,-0.28
2,2012-12-27,13096.31,13114.97,13141.74,12964.08,100.16,-0.14
3,2015-07-21,17919.29,18096.67,18096.67,17868.34,100.17,-1.00
4,2016-04-20,18096.27,18059.49,18167.63,18031.21,100.21,0.24


In [22]:
# read in store file
file2 = "store_update.csv"

store_df = pd.read_csv(file2)
store_df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/6/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [23]:
# convert header UC to LC
store_df.columns = store_df.columns.str.lower()
store_df

,row id,order id,order date,ship date,ship mode,customer id,customer name,segment,country,city,state,postal code,region,product id,category,sub-category,product name,sales
0,1,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/6/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,05/21/2017,28/05/2017,Standard Class,SH-19975,Sally Hughsby,Corporate,United States,Chicago,Illinois,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9796,9797,CA-2016-128608,12/1/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9797,9798,CA-2016-128608,12/1/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880
9798,9799,CA-2016-128608,12/1/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760


In [24]:
# change header names where needed
store_df = store_df.rename(columns={'row id':'row_id',
                                    'order id':'order_id',
                                    'order date' :'order_date',
                                    'ship date':'ship_date',
                                    'ship mode': 'ship_mode',
                                    'customer id': 'customer_id',
                                    'customer name': 'customer_name',
                                    'postal code': 'postal_code',
                                    'product id': 'product_id',
                                    'sub-category': 'sub_category',
                                    'product name': 'product_name'})
store_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales
0,1,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,8/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/6/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [25]:
store_df.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub_category',
       'product_name', 'sales'],
      dtype='object')

In [28]:
store_df=store_df[['order_id', 'order_date', 
                 'city', 'state','postal_code', 'region', 'product_id', 'category', 'sub_category','product_name', 'sales']]
store_df.head()

,order_id,order_date,city,state,postal_code,region,product_id,category,sub_category,product_name,sales
0,CA-2017-152156,2017-08-11,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,CA-2017-152156,2017-08-11,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,CA-2017-138688,2017-12-06,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,US-2016-108966,2016-11-10,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,US-2016-108966,2016-11-10,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [29]:
# convert "order_date" to pd datetime
store_df['order_date']= pd.to_datetime(store_df["order_date"], format='%m/%d/%Y', errors='coerce')
store_df.head(50)

,order_id,order_date,city,state,postal_code,region,product_id,category,sub_category,product_name,sales
0,CA-2017-152156,2017-08-11,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,CA-2017-152156,2017-08-11,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,CA-2017-138688,2017-12-06,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,US-2016-108966,2016-11-10,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,US-2016-108966,2016-11-10,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
5,CA-2015-115812,2015-09-06,Los Angeles,California,90032.0,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600
6,CA-2015-115812,2015-09-06,Los Angeles,California,90032.0,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.2800
7,CA-2015-115812,2015-09-06,Los Angeles,California,90032.0,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.1520
8,CA-2015-115812,2015-09-06,Los Angeles,California,90032.0,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.5040
9,CA-2015-115812,2015-09-06,Los Angeles,California,90032.0,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9000


In [30]:
# connect to pgAdmin
connection_String_format= '<user>:<passowrd>@<url>:<port>/<database_name>'
rds_connection_string = "postgres:Stuart2609!@localhost:5432/etl"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [31]:
engine.table_names()

['store', 'dow']

In [32]:
dow_df.to_sql(name='dow', con=engine, if_exists='append', index=False)

In [33]:
pd.read_sql_query('select * from dow', con=engine).tail()

,id,dt,price,open,high,low,vol_m,chng
2419,2420,2015-09-16,16739.95,16599.51,16755.98,16593.90,99.62,0.84
2420,2421,2015-12-07,17730.51,17845.49,17845.49,17639.25,99.67,-0.66
2421,2422,2013-09-12,15300.64,15327.14,15345.32,15283.26,99.76,-0.17
2422,2423,2013-02-08,13992.97,13944.05,14022.62,13944.05,99.86,0.35
2423,2424,2015-11-04,17867.58,17929.58,17964.12,17828.83,99.89,-0.28


In [35]:
store_df.to_sql(name='store', con=engine, if_exists='append', index=False)